In [1]:
import django_initializer
from telemetry.fast_lap_analyzer import FastLapAnalyzer
from telemetry.racing_stats import RacingStats
from telemetry.models import FastLap, Lap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots

import pandas as pd
import numpy as np

from telemetry.visualizer import *

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()
fast_lap_analyzer = FastLapAnalyzer()
racing_stats = RacingStats()

2023-07-06 16:49:36,404 DEBUG Influx: Connected to https://telemetry.b4mad.racing/


In [2]:
kwargs = {
    "game": "iRacing",
    # "track": "oschersleben gp",
    # "track": "aragon gp",
    "track": "jerez gp",
    "car": "Ferrari 488 GT3 Evo 2020",
    "valid": True,
}
laps = racing_stats.laps(**kwargs)
laps = laps[:10]
# laps = list(laps)
fast_lap_analyzer.laps = laps
data_frames, laps_with_telemetry = fast_lap_analyzer.fetch_lap_telemetry()

2023-07-06 16:49:36,503 DEBUG Influx: Connected to https://telemetry.b4mad.racing/
2023-07-06 16:49:36,508 INFO Fetching telemetry for iRacing - jerez gp - Ferrari 488 GT3 Evo 2020
2023-07-06 16:49:36,508 INFO   track.id 61 car.id 9
2023-07-06 16:49:36,509 INFO   session 1688310090 lap.id 176859 number 3
2023-07-06 16:49:36,509 INFO   length 4368 time 102.7625 valid True
2023-07-06 16:49:36,509 INFO   start 2023-07-02 17:04:59.217731+00:00 end 2023-07-02 17:06:42.026664+00:00
2023-07-06 16:49:37,042 INFO Fetching telemetry for iRacing - jerez gp - Ferrari 488 GT3 Evo 2020
2023-07-06 16:49:37,043 INFO   track.id 61 car.id 9
2023-07-06 16:49:37,043 INFO   session 1688223703 lap.id 173644 number 3
2023-07-06 16:49:37,043 INFO   length 4369 time 102.7718 valid True
2023-07-06 16:49:37,044 INFO   start 2023-07-01 17:05:13.753633+00:00 end 2023-07-01 17:06:56.538103+00:00
2023-07-06 16:49:37,559 INFO Fetching telemetry for iRacing - jerez gp - Ferrari 488 GT3 Evo 2020
2023-07-06 16:49:37,559

In [3]:
# influx_data_frames = []
# for df in data_frames:
#     influx_data_frames.append(df.copy())

# data_frames = []
# for df in influx_data_frames:
#     data_frames.append(df.copy())

In [4]:
current_sectors = fast_lap_analyzer.current_fast_lap_sectors()
display(current_sectors)

[{'start': 0, 'end': 322},
 {'start': 323, 'end': 747},
 {'start': 748, 'end': 1028},
 {'start': 1029, 'end': 1789},
 {'start': 1790, 'end': 2134},
 {'start': 2135, 'end': 2373},
 {'start': 2374, 'end': 2752},
 {'start': 2753, 'end': 3176},
 {'start': 3177, 'end': 3724},
 {'start': 3725, 'end': 4367}]

In [5]:
fig = lap_fig(data_frames[0], full_range=True, columns=["Throttle"])
fig.show()

In [6]:
# for i, df in enumerate(data_frames):
#     fig = lap_fig(df, full_range=True, columns=["Throttle"])
#     fig.show()
sector_start_end, df_max = fast_lap_analyzer.extract_sectors(data_frames)
display(sector_start_end)

# are the sectors similar to the current fast lap?
similarity = fast_lap_analyzer.similar_sectors(sector_start_end, current_sectors)
display(f"Sectors are similar: {similarity}")

fig = lap_fig(df_max, full_range=True, columns=["Throttle"])
# add a vertical line for each sector
for i, sector in enumerate(sector_start_end):
    fig_add_shape(fig, x0=sector["start"], x1=sector["start"], color="blue")
    fig_add_shape(fig, x0=sector["end"] - 1, x1=sector["end"] - 1, color="red")
fig.show()

2023-07-06 16:49:41,749 DEBUG Lap too short, skipping


[{'start': 0, 'end': 322.0, 'length': 322},
 {'start': 323.0, 'end': 747.0, 'length': 424},
 {'start': 748.0, 'end': 1029.0, 'length': 281},
 {'start': 1030.0, 'end': 1789.0, 'length': 759},
 {'start': 1790.0, 'end': 2134.0, 'length': 344},
 {'start': 2135.0, 'end': 2373.0, 'length': 238},
 {'start': 2374.0, 'end': 2752.0, 'length': 378},
 {'start': 2753.0, 'end': 3176.0, 'length': 423},
 {'start': 3177.0, 'end': 3724.0, 'length': 547},
 {'start': 3725.0, 'end': 4367.34, 'length': 642}]

2023-07-06 16:49:41,762 DEBUG start_diffs: [0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] med: 0.1


'Sectors are similar: True'

In [7]:
segments, used_laps = fast_lap_analyzer.extract_segments(sector_start_end, data_frames, laps_with_telemetry, df_max)
track_length = df_max["DistanceRoundTrack"].max()
for i in range(len(segments)):
    segment = segments[i]
    # display(segment.start, segment.end, segment.turn)
    # display(segment.telemetry)
    # df = telemetry_for_fig(segment)
    df = segment.telemetry
    fig = lap_fig(df, columns=["Throttle", "Brake"])
    brake_features = segment.brake_features()
    if brake_features:
        # brake_features = features_for_fig(segment, track_length, brake_features)
        fig_add_features(fig, brake_features)
    throttle_features = segment.throttle_features()
    if throttle_features:
        # throttle_features = features_for_fig(segment, track_length, throttle_features)
        fig_add_features(fig, throttle_features, color="green")
    display(f"time: {segment.time} {segment.track_length}")
    fig.show()

2023-07-06 16:49:41,888 ERROR sector 8 is empty
2023-07-06 16:49:41,897 ERROR sector 8 is empty
2023-07-06 16:49:41,907 ERROR sector 8 is empty
2023-07-06 16:49:41,917 ERROR sector 8 is empty


'time: 7.28606653 4368.34'

'time: 12.1166664 4368.34'

'time: 5.149999999999999 4368.34'

'time: 14.700000799999998 4368.34'

...

'time: 1.5333345000000023 4368.34'

'time: 7.8500023 4368.34'

'time: 10.916666200000002 4368.34'

'time: 14.26666999999999 4368.34'

'time: 15.483335499999995 4368.34'